In [25]:
%pylab inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import pandas as pd

df = pd.read_csv("salary-train.csv")
df_test = pd.read_csv("salary-test-mini.csv")

Populating the interactive namespace from numpy and matplotlib


Проведите предобработку:
* Приведите тексты к нижнему регистру (text.lower()).
* Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты.

In [26]:
def normalize_text(column):
    x = column.str.lower()
    return x.replace('[^a-zA-Z0-9]', ' ', regex=True)

Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=5)
text_vectors = vectorizer.fit_transform(normalize_text(df['FullDescription']))
test_text_vectors = vectorizer.transform(normalize_text(df_test['FullDescription']))

Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'.
Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

In [47]:
df['LocationNormalized'].fillna('nan', inplace=True)
df['ContractTime'].fillna('nan', inplace=True)
df_test['LocationNormalized'].fillna('nan', inplace=True)
df_test['ContractTime'].fillna('nan', inplace=True)

In [51]:
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
categ = enc.fit_transform(df[['LocationNormalized', 'ContractTime']].to_dict('records'))
test_categ = enc.transform(df_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

In [56]:
from scipy.sparse import hstack
X = hstack([text_vectors, categ])
X_test = hstack([test_text_vectors, test_categ])

Обучите гребневую регрессию с параметром alpha=1. Целевая переменная записана в столбце SalaryNormalized.

In [57]:
from sklearn.linear_model import Ridge

clf = Ridge(alpha=1.0)
clf.fit(X, df['SalaryNormalized']) 

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [62]:
clf.predict(X_test)

array([ 56571.82947114,  37188.39602618])

In [63]:
with open("submissions/4_1.txt", "w") as f:
    f.write("56571.83 37188.40")